In [1]:
using WAV
using Random
using FFTW

In [184]:
function fft_b(x)
    N = length(x)
    if log2(N) % 1 > 0
        throw(ArgumentError("must be a power of 2"))
    end
    N_min = min(N, 1)
    
    n = collect(0:(N_min-1))
    M = exp.(-2π * im * n * n'/ N_min)
    X = M * reshape(x, (:, N_min))'
    while size(X)[1] < N
        X_even = X[:, 1:Int(size(X)[2] / 2)]
        X_odd = X[:, (Int(size(X)[2] / 2)+1):end]
        t = exp.(-1*im * π * collect(0:(size(X)[1]-1))/size(X)[1])
        terms = hcat(map(x->t, 1:size(X_odd)[2])...)
        X = vcat(X_even + terms .* X_odd, X_even - terms .* X_odd)
    end
    return vec(X)
end
        
        
    

fft_b (generic function with 1 method)

In [185]:
@time new = fft_b([1,2,3,4, 5, 6, 7, 8])

  0.055075 seconds (735 allocations: 40.484 KiB)


8-element Array{Complex{Float64},1}:
                36.0 + 0.0im
                -4.0 + 9.65685424949238im
                -4.0 + 4.0im
                -4.0 + 1.6568542494923797im
                -4.0 + 0.0im
 -3.9999999999999996 - 1.6568542494923797im
 -3.9999999999999996 - 4.0im
 -3.9999999999999987 - 9.65685424949238im

In [186]:
@time old =fft([1,2,3,4, 5, 6, 7, 8])

  0.000083 seconds (36 allocations: 2.984 KiB)


8-element Array{Complex{Float64},1}:
 36.0 + 0.0im
 -4.0 + 9.65685424949238im
 -4.0 + 4.0im
 -4.0 + 1.6568542494923806im
 -4.0 + 0.0im
 -4.0 - 1.6568542494923806im
 -4.0 - 4.0im
 -4.0 - 9.65685424949238im

In [187]:
isapprox(new, old)

true

In [188]:
w = rand(16);

In [193]:
@time x = fft_b(w)

  0.000126 seconds (185 allocations: 14.281 KiB)


16-element Array{Complex{Float64},1}:
      5.67007062927372 + 0.0im
   -0.7613180247733727 - 1.9272603229745813im
    0.7225606777460523 + 0.1909678952047974im
 0.0064954194823636235 - 1.4228541890648465im
   -0.5086083857694561 - 0.04555230839112401im
   -0.4505390346312546 + 0.06572595212003618im
   0.21257882807633693 - 0.23939679469784292im
   -0.6778337618501359 + 0.7613226277852114im
   0.07131089398635293 + 0.0im
   -0.6778337618501357 - 0.7613226277852115im
   0.21257882807633693 + 0.23939679469784292im
  -0.45053903463125455 - 0.06572595212003607im
   -0.5086083857694561 + 0.04555230839112401im
  0.006495419482363152 + 1.422854189064847im
    0.7225606777460523 - 0.1909678952047974im
   -0.7613180247733732 + 1.927260322974581im

In [194]:
@time y = fft(w)

  0.000089 seconds (35 allocations: 3.094 KiB)


16-element Array{Complex{Float64},1}:
     5.67007062927372 + 0.0im
  -0.7613180247733731 - 1.9272603229745813im
   0.7225606777460524 + 0.1909678952047974im
 0.006495419482363346 - 1.422854189064847im
  -0.5086083857694561 - 0.04555230839112401im
 -0.45053903463125455 + 0.06572595212003618im
   0.2125788280763369 - 0.23939679469784292im
  -0.6778337618501358 + 0.7613226277852116im
  0.07131089398635293 + 0.0im
  -0.6778337618501358 - 0.7613226277852116im
   0.2125788280763369 + 0.23939679469784292im
 -0.45053903463125455 - 0.06572595212003618im
  -0.5086083857694561 + 0.04555230839112401im
 0.006495419482363346 + 1.422854189064847im
   0.7225606777460524 - 0.1909678952047974im
  -0.7613180247733731 + 1.9272603229745813im

In [191]:
isapprox(x, y)

true

In [192]:
w2 = rand(1024^2);

In [201]:
@time x2 = fft_b(w2);

  1.328822 seconds (1.28 k allocations: 1.508 GiB, 21.26% gc time)


In [202]:
@time y2 = fft(w2);

  0.095503 seconds (37 allocations: 32.003 MiB, 9.26% gc time)


In [198]:
isapprox(x2, y2)

true